In [12]:
import pickle
import mysql.connector
from flask import Flask, request, render_template
import numpy as np

application = Flask(__name__)
app = application

random = pickle.load(open('random.pkl', 'rb'))
standard = pickle.load(open('Standard.pkl', 'rb'))

def get_db_connection():
    connection = mysql.connector.connect(
        host="127.0.0.1",          
        user="root",              
        password="kshi180405",   
        database="employee_db"  
    )
    return connection
@app.route('/')
def home():
    return render_template('home.html')

@app.route('/index', methods=['GET', 'POST'])
def predict_datapoint():
    if request.method == 'POST':
        Department = int(request.form.get('Department'))
        Gender = int(request.form.get('Gender'))
        Age = int(request.form.get('Age'))
        Job_Title = float(request.form.get('Job_Title'))
        Education_Level = int(request.form.get('Education_Level'))
        Monthly_Salary = int(request.form.get('Monthly_Salary'))
        Work_Hours_Per_Week = int(request.form.get('Work_Hours_Per_Week'))
        Projects_Handled = int(request.form.get('Projects_Handled'))

        Overtime_Hours = int(request.form.get('Overtime_Hours'))
        Sick_Days = int(request.form.get('Sick_Days'))
        Remote_Work_Frequency = int(request.form.get('Remote_Work_Frequency'))
        Promotions=int(request.form.get('Promotions'))
        Employee_Satisfaction_Score=int(request.form.get('Employee_Satisfaction_Score'))

        # Scale the data
        new_data_scaled = standard.transform([[Department,Gender, Age, Job_Title, Education_Level,
                                               Monthly_Salary, Work_Hours_Per_Week,
                                             Projects_Handled, Overtime_Hours, Sick_Days,
                                             Remote_Work_Frequency, Promotions, Employee_Satisfaction_Score]])

        # Predict
        result = random.predict(new_data_scaled)
        Performance_Score = float(result[0])
                # Standardize input
        new_data_scaled = standard.transform([[
                Department, Gender, Age, Job_Title, Education_Level, Monthly_Salary,
                Work_Hours_Per_Week, Projects_Handled, Overtime_Hours, Sick_Days,
                Remote_Work_Frequency, Promotions, Employee_Satisfaction_Score
            ]])

            # Predict
        result = random.predict(new_data_scaled)
        Performance_Score = float(result[0])

            # (Optional) Save to DB
        conn = get_db_connection()
        cursor = conn.cursor()
        insert_query = """
            INSERT INTO predictions (
                Department, Gender, Age, Job_Title, Education_Level,
                    Monthly_Salary, Work_Hours_Per_Week, Projects_Handled,
                    Overtime_Hours, Sick_Days, Remote_Work_Frequency,
                    Promotions, Employee_Satisfaction_Score, Performance_Score
                )
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """
        data = (Department, Gender, Age, Job_Title, Education_Level,
                    Monthly_Salary, Work_Hours_Per_Week, Projects_Handled,
                    Overtime_Hours, Sick_Days, Remote_Work_Frequency,
                    Promotions, Employee_Satisfaction_Score, Performance_Score)
        cursor.execute(insert_query, data)
        conn.commit()
        cursor.close()
        conn.close()
       
        

        return render_template('index.html', results=f"Predicted Score: {Performance_Score:.2f}")

    else:
        return render_template('index.html')


if __name__ == "__main__":
    app.run(host='0.0.0.0')


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.187.74.156:5000


Press CTRL+C to quit
10.187.74.156 - - [16/Oct/2025 08:16:23] "GET /index HTTP/1.1" 200 -
f:\anacoda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
f:\anacoda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
10.187.74.156 - - [16/Oct/2025 08:16:51] "POST /index HTTP/1.1" 200 -
